# Collecte des observations d'écoulement d'une année avec Hub'Eau
*Antonio Andrade, ingénieur de données, [Office français de la biodiversité](https://ofb.gouv.fr/)*  

[Hub'Eau](https://hubeau.eaufrance.fr/) est une plateforme nationale d'API REST permettant d'automatiser l'accès et le traitement des données ouvertes du [Sytème d'Information sur l'Eau (SIE)](https://www.eaufrance.fr/le-systeme-dinformation-sur-leau-sie) Ce tutoriel illustre l'utilisation de l'[API Ecoulement des cours d'eau](https://hubeau.eaufrance.fr/page/api-ecoulement#/) pour télécharger les observations d'écoulement réalisées par les agents de l'OFB et bancarisées dans l'observatoire national des étiages, [Onde](https://onde.eaufrance.fr/).

## Chargement des modules
Commençons par charger en mémoire les modules python nécessaires pour nos traitements :
- le module `json` pour traiter des données au format json
- le module standard `os` pour interagir avec le système d'exploitation
- le module standard `requests` pour interagir avec les API REST Hub'Eau
- le module standard `time` pour temporiser l'envoi des requêtes

In [1]:
# Chargement des modules nécessaires à nos traitements
import json
import os
import requests
import time

import geopandas
import pandas

print("Chargement OK")

Chargement OK


## Paramétrage du téléchargement
Poursuivons avec le paramétrage de notre traitement :

In [2]:
# Point d'accès de l'API Hub'Eau Prélèvements en eau
endpoint = "v1/ecoulement"
# Opération correspondant aux chroniques d'observation
operation = "observations"

# Année d'observation
year = 2022
# Données attendues dans la réponse (cf. données disponibles du modèle Observation)
fields = [
    "code_station",
    "libelle_station",
    "uri_station",
    "code_departement",
    "libelle_departement",
    "code_commune",
    "libelle_commune",
    "code_region",
    "libelle_region",
    "code_bassin",
    "libelle_bassin",
    "coordonnee_x_station",
    "coordonnee_y_station",
    "code_projection_station",
    "libelle_projection_station",
    "code_cours_eau",
    "libelle_cours_eau",
    "uri_cours_eau",
    "code_campagne",
    "code_reseau",
    "libelle_reseau",
    "uri_reseau",
    "date_observation",
    "code_ecoulement",
    "libelle_ecoulement",
    "latitude",
    "longitude"
]

print("Paramétrage OK")

Paramétrage OK


## Définition d'une classe d'interfaçage avec Hub'Eau
Construisons une classe Hubeau d'interçage avec Hub'Eau :

In [3]:
class Hubeau:
    
    def __init__(self, response_format="json", page_size=1000, max_try=5, delay_before_request=3):
        # URL de base des API Hub'Eau
        self._base = "http://hubeau.eaufrance.fr/api"
        # Format des résultats
        self._response_format = response_format
        # Nombre de résultats par page
        self._page_size = page_size
        # Nombre maximum de tentatives d'interrogation d'une API
        self._max_try = max_try
        # Nombre maximum de résultats pour une recherche
        self.max_results = 20000
        # Délai entre deux requêtes
        self._delay_before_request = delay_before_request
        
    @property
    def response_format(self):
        """The format of the response : json, geojson, csv."""
        return self._response_format

    @response_format.setter
    def page_size(self, value):
        self._page_size = value
    
    @property
    def page_size(self):
        """The result page size : 0 to 5000 depending of the API."""
        return self._page_size
    
    @page_size.setter
    def page_size(self, value):
        self._page_size = value
    
    @property
    def max_try(self):
        """The number of tries before giving up."""
        return self._max_try
    
    @max_try.setter
    def max_try(self, value):
        self._max_try = value
    
    @property
    def delay_before_request(self):
        """The delay in seconds between requests."""
        return self._delay_before_request

    @delay_before_request.setter
    def delay_before_request(self, value):
        self._delay_before_request = value

    def _build_url(self, endpoint, operation, parameters):
        """Construit l'url de recherche
        """
        if "format" not in parameters:
            parameters["format"] = self.response_format
        if "size" not in parameters:
            parameters["size"] = self.page_size
        
        # Modification de l'operation en cas d'oubli de l'extension .csv
        ext = ".csv" if self.response_format == "csv" and operation[-4:] != ".csv" else ""
        
        params = "&".join([f"{key}={value}" for key, value in parameters.items()])
        return f"{self._base}/{endpoint}/{operation}{ext}?{params}"
    
    def _request(self, url):
        """Renvoie la réponse à une requête Hub'eau
        @param url : URL Hub'eau requêtée
        """
        print(f"{url}")
        request_number = 0
        while request_number < self.max_try:
            try:
                # Interrogation de l'API Hub'Eau
                response = requests.get(url)
                request_number += 1
                print(f"\t=> Tentative {request_number} - Statut {response.status_code} ({response.reason})")

                # Gestion des erreurs d'interrogation
                if response.ok:
                    break

                # Temporisation avant toute nouvelle requête
                time.sleep(self.delay_before_request)
            except Exception as e:
                print(f"=> Erreur \n{e}")

        if request_number == self.max_try:
            print(f"\t=> Abandon après {request_number} tentatives")
            return None
        
        return response
    
    def _search_json(self, url):
        """Renvoie les résultats d'une recherche au format json"""
        # Initialisation de la liste de résultats de recherche
        results = []
        # Initialisation de l'url de la 1ère page à requêter
        next_url = f"{url}&page=1"

        # Récupération des résultats page par page
        while True:

            # Récupération de la ième page de résultats
            response = self._request(next_url)
            if response is None:
                # Erreur d'interrogation
                return []
            
            try:
                # Récupération de la réponse au format json
                response_json = response.json()
            except:
                # Erreur de conversion au format json
                return []
            
            # Récupération des données métier
            results += response_json['data'] if self.response_format == "json" else response_json['features']

            # Dernière page de données traitée
            if response_json['next'] is None or len(results) >= self.max_results:
                if response_json['next'] is not None:
                    print(f"Le résultat de la recherche est limité aux {self.max_results} premières données")
                break
            
            # URL de la page suivante de résultats
            next_url = response_json['next']
            
            # Re paramétrage du nombre de résultats de la dernière page
            if (len(results) + self.page_size) >= self.max_results:
                next_url = next_url.replace(
                    f"size={self.page_size}", "size={}".format(self.max_results - len(results))
                )
        
        return results
    
    def _search_csv(self, url):
        """Renvoie les résultats d'une recherche au format csv"""
        # Récupération des résultats de la page unique de résultats
        response = self._request(url)
        if response is None:
            # Erreur d'interrogation
            return []
        # Renvoie une liste de lignes de données
        return str(response.text).split("\n")
    
    def _build_response(self, data):
        """Construit la réponse à une recherche
        @param data : liste des résultats de la recherche
        """
        if self.response_format == "geojson":
            feature_col = {
                "type": "FeatureCollection",
                "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}},
                "features": data
            }
            # Création d'un GeoDataFrame
            return geopandas.GeoDataFrame.from_features(feature_col)
        elif self.response_format == "csv":
            columns = str(data[0]).split(";")
            columns = [column.replace('"', "") for column in columns]
            data = [line.split(";") for line in data]
            tmp = []
            for line in data:
                tmp.append([field.replace('"', "") for field in line])
            data = tmp
            return pandas.DataFrame(data[1:], columns=columns)
        else:
            # Création d'un DataFrame
            return pandas.DataFrame(data)
        
    def search(self, endpoint, operation, parameters):
        """Renvoie les résultats d'une recherche
        @param endpoint: string # Point d'accès de l'API
        @param operation: string # Opération de l'API
        """
        # Construction de l'url de recherche
        url = self._build_url(endpoint, operation, parameters)
        
        data = []
        if self.response_format in ["json", "geojson"]:
            data = self._search_json(url)
        elif self.response_format == "csv":
            data = self._search_csv(url)
        
        return self._build_response(data)

## Recherche des observations d'une année
Utilisons la classe Hubeau pour récupérer les observations de l'année spécifiée:

In [4]:
parameters = {
    "date_observation_min": f"{year}-01-01",
    "date_observation_max": f"{year}-12-31",
    "fields": ",".join(fields)
}

hubeau = Hubeau(response_format="json")
gdf = hubeau.search(endpoint, operation, parameters)

http://hubeau.eaufrance.fr/api/v1/ecoulement/observations?date_observation_min=2022-01-01&date_observation_max=2022-12-31&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=json&size=1000&page=1
	=> Tentative 1 - Statut 206 (Partial Content)
https://hubeau.eaufrance.fr/api/v1/ecoulement/observations?date_observation_min=2022-01-01&date_observation_max=2022-12-31&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projec

	=> Tentative 1 - Statut 206 (Partial Content)
https://hubeau.eaufrance.fr/api/v1/ecoulement/observations?date_observation_min=2022-01-01&date_observation_max=2022-12-31&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=json&page=16&size=1000
	=> Tentative 1 - Statut 206 (Partial Content)
https://hubeau.eaufrance.fr/api/v1/ecoulement/observations?date_observation_min=2022-01-01&date_observation_max=2022-12-31&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee

In [5]:
gdf

,code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,...,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,longitude,latitude
0,H2051021,Brêche,http://id.eaufrance.fr/SiteHydro/H2051021,60,Oise,60535,REUIL-SUR-BRECHE,32,Hauts-de-France,03,...,http://id.eaufrance.fr/CoursEau_Carthage2017/H...,97017,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-12-27,3,Assec,2.220306,49.519642
1,H2062021,Arré,http://id.eaufrance.fr/SiteHydro/H2062021,60,Oise,60581,SAINT-JUST-EN-CHAUSSEE,32,Hauts-de-France,03,...,http://id.eaufrance.fr/CoursEau_Carthage2017/H...,97017,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-12-27,2,Ecoulement non visible,2.439828,49.505398
2,H0360001,Aronde,http://id.eaufrance.fr/SiteHydro/H0360001,60,Oise,60418,MONTIERS,32,Hauts-de-France,03,...,http://id.eaufrance.fr/CoursEau_Carthage2017/H...,97017,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-12-27,1,Ecoulement visible,2.596089,49.502327
3,E6407541,Avre,http://id.eaufrance.fr/SiteHydro/E6407541,60,Oise,60035,AVRICOURT,32,Hauts-de-France,01,...,http://id.eaufrance.fr/CoursEau_Carthage2017/E...,97017,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-12-27,3,Assec,2.859155,49.652985
4,H0321042,Ru des Prés de Vienne,http://id.eaufrance.fr/SiteHydro/H0321042,60,Oise,60499,PLESSIS-DE-ROYE,32,Hauts-de-France,03,...,http://id.eaufrance.fr/CoursEau_Carthage2017/H...,97017,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-12-27,1,Ecoulement visible,2.820128,49.583894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,J1205411,Le Guinguenoual,http://id.eaufrance.fr/SiteHydro/J1205411,22,Côtes-d'Armor,22268,RUCA,53,Bretagne,04,...,http://id.eaufrance.fr/CoursEau_Carthage2017/J...,96404,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-07-16,1a,Ecoulement visible acceptable,-2.356847,48.555415
19996,J7300001,Le Meu,http://id.eaufrance.fr/SiteHydro/J7300001,22,Côtes-d'Armor,22371,TREMOREL,53,Bretagne,04,...,http://id.eaufrance.fr/CoursEau_Carthage2017/J...,96404,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-07-16,2,Ecoulement non visible,-2.287123,48.204250
19997,J1140001,Le Guébriant,http://id.eaufrance.fr/SiteHydro/J1140001,22,Côtes-d'Armor,22237,PLUDUNO,53,Bretagne,04,...,http://id.eaufrance.fr/CoursEau_Carthage2017/J...,96404,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-07-16,2,Ecoulement non visible,-2.285387,48.533895
19998,J1105811,Le Quiloury,http://id.eaufrance.fr/SiteHydro/J1105811,22,Côtes-d'Armor,22185,PLENEE-JUGON,53,Bretagne,04,...,http://id.eaufrance.fr/CoursEau_Carthage2017/J...,96404,0000000134,Onde OFB,http://id.eaufrance.fr/dc/0000000134,2022-07-16,2,Ecoulement non visible,-2.437071,48.367069


In [70]:
codes_region = [
    11, # Ile-de-France
    24, # Centre-Val de Loire
    27, # Bourgogne-Franche-Comté
    28, # Normandie
    32, # Hauts-de-France
    44, # Grand Est
    52, # Pays de la Loire
    53, # Bretagne
    75, # Nouvelle-Aquitaine
    76, # Occitanie
    84, # Auvergne-Rhône-Alpes
    93, # Provence-Alpes-Côte D'Azur
    94, # Corse
]
# Initialisation de l'interface Hub'Eau
hubeau = Hubeau(response_format="geojson")
gdf = None
for code_region in codes_region:
    # Récupération des observations d'un couple (année, région)
    parameters = {
        "date_observation_min": f"{year}-01-01",
        "date_observation_max": f"{year}-12-31",
        "code_region": ",".join([str(code_region)]),
        "fields": ",".join(fields)
    }
    if gdf is None:
        gdf = hubeau.search(endpoint, operation, parameters)
    gdf2 = hubeau.search(endpoint, operation, parameters)
    gdf = pandas.concat([gdf, gdf2])

# Réindexation du DataFrame
gdf.reset_index(drop=True, inplace=True)
# Enregistrement des données
gdf.to_file(os.path.join(os.getcwd(), f"onde_observations_{year}.gpkg"), driver='GPKG', layer='onde_observations')

http://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=11&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=geojson&size=1000
	=> Tentative 1 - Statut 200 (OK)
http://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=11&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_stat

https://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=52&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=geojson&page=2&size=1000
	=> Tentative 1 - Statut 206 (Partial Content)
http://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=53&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station

http://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=84&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=geojson&size=1000
	=> Tentative 1 - Statut 206 (Partial Content)
https://hubeau.eaufrance.fr/api/vbeta/ecoulement/observations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=84&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_p

## Enregistrement des résultats
Terminons notre traitement avec l'enregistrement des résultats :

In [15]:
# Sauvegarde des résultats
#with open(os.path.join(os.getcwd(), f"onde_observations_{year}.geojson"), "w", encoding="utf8") as out_fic:
#    json.dump(response, out_fic)

gdf.to_file(os.path.join(os.getcwd(), f"onde_observations_{year}.geojson"), driver='GeoJSON', encoding="utf-8")
gdf.to_file(os.path.join(os.getcwd(), f"onde_observations_{year}.gpkg"), driver='GPKG', layer='onde_observations')
gdf.to_file(os.path.join(os.getcwd(), f"onde_observations_{year}.shp"))

In [71]:
# Récupération des stations d'observation
operation = "stations"

hubeau = Hubeau(response_format="geojson")
gdf = hubeau.search(endpoint, operation, parameters)

# Enregistrement des données
gdf.to_file(os.path.join(os.getcwd(), f"onde_stations.gpkg"), driver='GPKG', layer='onde_stations')

http://hubeau.eaufrance.fr/api/vbeta/ecoulement/stations?date_observation_min=2020-01-01&date_observation_max=2020-12-31&code_region=94&fields=code_station,libelle_station,uri_station,code_departement,libelle_departement,code_commune,libelle_commune,code_region,libelle_region,code_bassin,libelle_bassin,coordonnee_x_station,coordonnee_y_station,code_projection_station,libelle_projection_station,code_cours_eau,libelle_cours_eau,uri_cours_eau,code_campagne,code_reseau,libelle_reseau,uri_reseau,date_observation,code_ecoulement,libelle_ecoulement,latitude,longitude&format=geojson&size=1000
	=> Tentative 1 - Statut 200 (OK)


Merci pour votre attention

Références :
- [Hub'Eau](https://hubeau.eaufrance.fr/)
- [BNPE](https://bnpe.eaufrance.fr/)
- [pandas](https://pandas.pydata.org/)